In [ ]:
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import multiprocessing
import nltk
nltk.download('stopwords')




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
csv_path= r'/content/drive/MyDrive/ic/hotel-reviews.csv'
df = pd.read_csv(csv_path)

In [ ]:
def clean_data(text):
    text = re.sub(r'[^ \nA-Za-z0-9À-ÖØ-öø-ÿ/]+', '', text)
    text = re.sub(r'[\\/×\^\]\[÷]', '', text)
    return text

In [ ]:
def change_lower(text):
    text = text.lower()
    return text

In [ ]:
stopwords_list = stopwords.words("english")
def remover(text):
    text_tokens = text.split(" ")
    final_list = [word for word in text_tokens if not word in stopwords_list]
    text = ' '.join(final_list)
    return text

In [ ]:
def get_w2vdf(df):
    w2v_df = pd.DataFrame(df["Description"]).values.tolist()
    for i in range(len(w2v_df)):
        w2v_df[i] = w2v_df[i][0].split(" ")
    return w2v_df

In [ ]:
def train_w2v(w2v_df):
    cores = multiprocessing.cpu_count()
    w2v_model = Word2Vec(min_count=4,
                         window=4,
                         size=300, 
                         alpha=0.03, 
                         min_alpha=0.0007, 
                         sg = 1,
                         workers=cores-1)
    
    w2v_model.build_vocab(w2v_df, progress_per=10000)
    w2v_model.train(w2v_df, total_examples=w2v_model.corpus_count, epochs=100, report_delay=1)
    return w2v_model

In [ ]:
df[["Description"]] = df[["Description"]].astype(str)
df["Description"] = df["Description"].apply(change_lower)
df["Description"] = df["Description"].apply(clean_data)
df["Description"] = df["Description"].apply(remover)

w2v_df = get_w2vdf(df)
w2v_model = train_w2v(w2v_df)


In [ ]:
df.shape

(29404, 4)

In [ ]:
w2v_model.wv.most_similar("bed")

[('beds', 0.6703979969024658),
 ('sofa', 0.4813436269760132),
 ('room', 0.478606641292572),
 ('bathroom', 0.47667816281318665),
 ('desktable', 0.4722157418727875),
 ('pillows', 0.4597439169883728),
 ('mattress', 0.4367106258869171),
 ('sheets', 0.4356108009815216),
 ('comfy', 0.4229239523410797),
 ('comfortable', 0.4076572060585022)]